In [2]:
import os
import pandas as pd
from binance.client import Client
from binance import ThreadedWebsocketManager
from datetime import datetime, timedelta


In [3]:
API_KEY = os.getenv('TEST_KEY')
API_SECRET = os.getenv('TEST_SECRET')
API_KEY

'3aH1Gh3xf1fsOxDEPpDERmBZ1NXSryFeWx3obCBDD3JveNIvoKHWKRvW3AktNnS2'

In [4]:
# Ensure that testnet is set to true !
client = Client(api_key = API_KEY, api_secret = API_SECRET, tld = "us", testnet= True)

In [5]:
client.get_account() #Test account details

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1672685531162,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [6]:
account = client.get_account()
account["accountType"]  #Account Type

'SPOT'

In [7]:
def get_history(symbol, interval, start, end = None):
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = start, end_str = end, limit = 1000)  # get the historical Klines with defined intervals 
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms") #make the first column of unix time into meaningful timestamps
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "Quote Asset Volume", 
                      "Number of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]  # rename columns for what we are going to use them
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy() #take only the columns we care about
        df.set_index("Date", inplace = True) # make the date column the index column
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
            
        return df

In [8]:
df = pd.DataFrame(account["balances"]) # extract balances from account
df

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [9]:
# Convert numbers to floats
df["free"] = pd.to_numeric(df["free"], errors = "coerce")
df["locked"] = pd.to_numeric(df["locked"], errors = "coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   asset   8 non-null      object 
 1   free    8 non-null      float64
 2   locked  8 non-null      float64
dtypes: float64(2), object(1)
memory usage: 320.0+ bytes


In [10]:
#Get prices for all testnet pairs
client.get_all_tickers()

[{'symbol': 'BNBBUSD', 'price': '245.70000000'},
 {'symbol': 'BTCBUSD', 'price': '16653.49000000'},
 {'symbol': 'ETHBUSD', 'price': '1212.67000000'},
 {'symbol': 'LTCBUSD', 'price': '75.49000000'},
 {'symbol': 'TRXBUSD', 'price': '0.05480000'},
 {'symbol': 'XRPBUSD', 'price': '0.34320000'},
 {'symbol': 'BNBUSDT', 'price': '245.70000000'},
 {'symbol': 'BTCUSDT', 'price': '16659.45000000'},
 {'symbol': 'ETHUSDT', 'price': '1213.09000000'},
 {'symbol': 'LTCUSDT', 'price': '75.51000000'},
 {'symbol': 'TRXUSDT', 'price': '0.05482000'},
 {'symbol': 'XRPUSDT', 'price': '0.34350000'},
 {'symbol': 'BNBBTC', 'price': '0.01475200'},
 {'symbol': 'ETHBTC', 'price': '0.07282000'},
 {'symbol': 'LTCBTC', 'price': '0.00453300'},
 {'symbol': 'TRXBTC', 'price': '0.00000329'},
 {'symbol': 'XRPBTC', 'price': '0.00002060'},
 {'symbol': 'LTCBNB', 'price': '0.30760000'},
 {'symbol': 'TRXBNB', 'price': '0.00022280'},
 {'symbol': 'XRPBNB', 'price': '0.00139300'}]

In [11]:
last24 = client.get_ticker(symbol = "BTCUSDT") #24 Price change statistic
last24

{'symbol': 'BTCUSDT',
 'priceChange': '-43.00000000',
 'priceChangePercent': '-0.257',
 'weightedAvgPrice': '16692.01695955',
 'prevClosePrice': '16702.28000000',
 'lastPrice': '16659.45000000',
 'lastQty': '0.17167500',
 'bidPrice': '16659.17000000',
 'bidQty': '0.14166400',
 'askPrice': '16659.40000000',
 'askQty': '0.15426800',
 'openPrice': '16702.45000000',
 'highPrice': '60000.00000000',
 'lowPrice': '4986.46000000',
 'volume': '7293.39896900',
 'quoteVolume': '121741539.28330063',
 'openTime': 1672706839255,
 'closeTime': 1672793239255,
 'firstId': 2255303,
 'lastId': 2364208,
 'count': 108906}

In [12]:
pd.to_datetime(last24["openTime"], unit = "ms") # Show datetime in a readable format

Timestamp('2023-01-03 00:47:19.255000')

In [13]:
timestamp = client._get_earliest_valid_timestamp(symbol = "BTCUSDT", interval = "1d") #earliest available data will reset every month
pd.to_datetime(timestamp, unit = "ms") # Show datetime in a readable format

Timestamp('2022-12-14 00:00:00')

In [14]:
df = get_history(symbol = "BTCUSDT", interval = "1d", start = timestamp)
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-12-14,17837.75,50000.00,11960.00,17795.18,4428.098892
2022-12-15,17795.18,18952.74,16831.78,17357.11,7740.685182
2022-12-16,17357.49,17525.00,16342.69,16630.42,8635.080110
2022-12-17,16630.72,17376.90,5312.94,16776.19,7435.010972
2022-12-18,16776.61,17134.31,5000.00,16738.62,6363.810820
2022-12-19,16738.61,16907.00,10000.00,16439.51,7505.842789
2022-12-20,16439.51,17055.56,6000.00,16896.24,8441.694337
2022-12-21,16896.19,17095.64,5000.00,16826.39,6965.755353
2022-12-22,16825.18,16867.03,4001.00,16821.02,7266.582324


In [15]:
def stream_data(msg):
    '''define how to process incoming websocket messages'''
    time = pd.to_datetime(msg["E"], unit="ms")
    price = msg["c"]
    
    print("Time: {} | Price: {}".format(time, price))

In [16]:
#initialize and start the websocket
twm = ThreadedWebsocketManager(tld = "us")
twm.start()

In [19]:
twm.start_symbol_miniticker_socket(callback = stream_data, symbol = "BTCUSDT")

'btcusdt@miniTicker'

In [20]:
twm.stop() #stop the websocket/stream

# Place a market Buy Order

In [21]:
#Buy 0.1 Bitcoin with USDT
order = client.create_order(symbol = "BTCUSDT", side = "BUY", type = "MARKET", quantity = 0.1)

In [22]:
order

{'symbol': 'BTCUSDT',
 'orderId': 12196506,
 'orderListId': -1,
 'clientOrderId': 'PjFwpJhJD0TLDPvUjYTXrl',
 'transactTime': 1672793284414,
 'price': '0.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '1665.93800000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1672793284414,
 'fills': [{'price': '16659.38000000',
   'qty': '0.10000000',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 2364247}],
 'selfTradePreventionMode': 'NONE'}

In [23]:
df = pd.DataFrame(order["fills"])
df

,price,qty,commission,commissionAsset,tradeId
0,16659.38000000,0.10000000,0.00000000,BTC,2364247


In [24]:
num_columns = ["price","qty","commission"]

In [28]:
for column in num_columns:
    df[column] = pd.to_numeric(df[column], errors = "coerce")

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 0 to commission
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            1 non-null      float64
 1   qty              1 non-null      float64
 2   commission       1 non-null      float64
 3   commissionAsset  1 non-null      object 
 4   tradeId          1 non-null      float64
dtypes: float64(4), object(1)
memory usage: 192.0+ bytes


In [31]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1672793284414,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.10000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '8334.06200000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [33]:
#Buy Bitcoin with 1000 USDT
order = client.create_order(symbol = "BTCUSDT", side = "BUY", type = "MARKET", quoteOrderQty = 1000)

In [34]:
order

{'symbol': 'BTCUSDT',
 'orderId': 12200478,
 'orderListId': -1,
 'clientOrderId': 'tSA6joXkNqHjbOGHjXf8mn',
 'transactTime': 1672793894642,
 'price': '0.00000000',
 'origQty': '0.06002100',
 'executedQty': '0.06002100',
 'cummulativeQuoteQty': '999.99307512',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'BUY',
 'workingTime': 1672793894642,
 'fills': [{'price': '16660.72000000',
   'qty': '0.06002100',
   'commission': '0.00000000',
   'commissionAsset': 'BTC',
   'tradeId': 2364951}],
 'selfTradePreventionMode': 'NONE'}

In [37]:
# Check the average price we paid per bitcoin in this transaction by checking how mych we paid vs how much we got
avr_price = float(order["cummulativeQuoteQty"]) / float(order["origQty"])
avr_price

16660.72

# Place a Market Sell Order

In [39]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1672793894642,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.16002100', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '7334.06892488', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [40]:
order = client.create_order(symbol = "BTCUSDT", side = "SELL", type = "MARKET", quantity = 0.16002100)

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1672794241876,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.71007560', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [41]:
order

{'symbol': 'BTCUSDT',
 'orderId': 12202491,
 'orderListId': -1,
 'clientOrderId': 'fQHFFgAG5uQpQmoArF3z1R',
 'transactTime': 1672794241876,
 'price': '0.00000000',
 'origQty': '0.16002100',
 'executedQty': '0.16002100',
 'cummulativeQuoteQty': '2666.64115072',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1672794241876,
 'fills': [{'price': '16664.32000000',
   'qty': '0.16002100',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 2365338}],
 'selfTradePreventionMode': 'NONE'}

In [42]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1672794241876,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.71007560', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

# Limit Orders

In [55]:
client.get_symbol_ticker(symbol = "BTCUSDT")

{'symbol': 'BTCUSDT', 'price': '16701.80000000'}

In [45]:
# Create Limit Buy Order that is Good to Call
order1 = client.create_order(symbol = "BTCUSDT", side = "BUY", type = "LIMIT", quantity = 0.1, 
                             timeInForce = "GTC", price = 16600)

In [59]:
# Create Limit Sell Order that is Good to Call
order2 = client.create_order(symbol = "BTCUSDT", side = "SELL", type = "LIMIT", quantity = 0.1, 
                             timeInForce = "GTC", price = 17000)

In [52]:
order1

{'symbol': 'BTCUSDT',
 'orderId': 12206187,
 'orderListId': -1,
 'clientOrderId': 'tnSdZuZ06BMA0CUtTGrwF0',
 'transactTime': 1672794807908,
 'price': '16700.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'workingTime': 1672794807908,
 'fills': [],
 'selfTradePreventionMode': 'NONE'}

In [ ]:
order2

In [61]:
order_id_buy = order1["orderId"]
order_id_sell = order2["orderId"]

In [49]:
client.get_order(symbol = "BTCUSDT", orderId = order_id_buy)

{'symbol': 'BTCUSDT',
 'orderId': 12204625,
 'orderListId': -1,
 'clientOrderId': '1IQ0qI2nKpKQiqpWE1qenG',
 'price': '16600.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'NEW',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'BUY',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1672794551542,
 'updateTime': 1672794551542,
 'isWorking': True,
 'workingTime': 1672794551542,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'NONE'}

In [56]:
client.get_order(symbol = "BTCUSDT", orderId = order_id_sell)

{'symbol': 'BTCUSDT',
 'orderId': 12206187,
 'orderListId': -1,
 'clientOrderId': 'tnSdZuZ06BMA0CUtTGrwF0',
 'price': '16700.00000000',
 'origQty': '0.10000000',
 'executedQty': '0.10000000',
 'cummulativeQuoteQty': '1670.00000000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'stopPrice': '0.00000000',
 'icebergQty': '0.00000000',
 'time': 1672794807908,
 'updateTime': 1672794820282,
 'isWorking': True,
 'workingTime': 1672794807908,
 'origQuoteOrderQty': '0.00000000',
 'selfTradePreventionMode': 'NONE'}

In [70]:
open_orders = client.get_open_orders(symbol = "BTCUSDT")

In [71]:
open_orders

[]

In [72]:
df = pd.DataFrame(open_orders)
df

""


# Cancel Open Orders

In [73]:
result = client.cancel_order(symbol = "BTCUSDT", orderId = order_id_buy)
result

BinanceAPIException: APIError(code=-2011): Unknown order sent.

In [74]:
result = client.cancel_order(symbol = "BTCUSDT", orderId = order_id_sell)
result

BinanceAPIException: APIError(code=-2011): Unknown order sent.

# Get Order History

In [75]:
all_orders = client.get_all_orders(symbol = "BTCUSDT")
all_orders

[{'symbol': 'BTCUSDT',
  'orderId': 12196506,
  'orderListId': -1,
  'clientOrderId': 'PjFwpJhJD0TLDPvUjYTXrl',
  'price': '0.00000000',
  'origQty': '0.10000000',
  'executedQty': '0.10000000',
  'cummulativeQuoteQty': '1665.93800000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1672793284414,
  'updateTime': 1672793284414,
  'isWorking': True,
  'workingTime': 1672793284414,
  'origQuoteOrderQty': '0.00000000',
  'selfTradePreventionMode': 'NONE'},
 {'symbol': 'BTCUSDT',
  'orderId': 12200478,
  'orderListId': -1,
  'clientOrderId': 'tSA6joXkNqHjbOGHjXf8mn',
  'price': '0.00000000',
  'origQty': '0.06002100',
  'executedQty': '0.06002100',
  'cummulativeQuoteQty': '999.99307512',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1672793894642,
  'updateTime': 1672793894642,
  '

In [76]:
pd.DataFrame(all_orders)

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,BTCUSDT,12196506,-1,PjFwpJhJD0TLDPvUjYTXrl,0.00000000,0.10000000,0.10000000,1665.93800000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1672793284414,1672793284414,True,1672793284414,0.00000000,NONE
1,BTCUSDT,12200478,-1,tSA6joXkNqHjbOGHjXf8mn,0.00000000,0.06002100,0.06002100,999.99307512,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,1672793894642,1672793894642,True,1672793894642,1000.00000000,NONE
2,BTCUSDT,12202491,-1,fQHFFgAG5uQpQmoArF3z1R,0.00000000,0.16002100,0.16002100,2666.64115072,FILLED,GTC,MARKET,SELL,0.00000000,0.00000000,1672794241876,1672794241876,True,1672794241876,0.00000000,NONE
3,BTCUSDT,12204625,-1,1IQ0qI2nKpKQiqpWE1qenG,16600.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,BUY,0.00000000,0.00000000,1672794551542,1672796261478,True,1672794551542,0.00000000,NONE
4,BTCUSDT,12206187,-1,tnSdZuZ06BMA0CUtTGrwF0,16700.00000000,0.10000000,0.10000000,1670.00000000,FILLED,GTC,LIMIT,SELL,0.00000000,0.00000000,1672794807908,1672794820282,True,1672794807908,0.00000000,NONE
5,BTCUSDT,12206887,-1,PA6VVagmuNmy6La4uCYGBi,17000.00000000,0.10000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,SELL,0.00000000,0.00000000,1672794927598,1672796262174,True,1672794927598,0.00000000,NONE


In [77]:
trades = client.get_my_trades(symbol = "BTCUSDT")

In [78]:
pd.DataFrame(trades)

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,BTCUSDT,2364247,12196506,-1,16659.38000000,0.10000000,1665.93800000,0.00000000,BTC,1672793284414,True,False,True
1,BTCUSDT,2364951,12200478,-1,16660.72000000,0.06002100,999.99307512,0.00000000,BTC,1672793894642,True,False,True
2,BTCUSDT,2365338,12202491,-1,16664.32000000,0.16002100,2666.64115072,0.00000000,USDT,1672794241876,False,False,True
3,BTCUSDT,2366432,12206187,-1,16700.00000000,0.09160900,1529.87030000,0.00000000,USDT,1672794820272,False,True,True
4,BTCUSDT,2366433,12206187,-1,16700.00000000,0.00839100,140.12970000,0.00000000,USDT,1672794820282,False,True,True
